# Anonimización de datos

La información de identificación personal (PII) es cualquier dato que pueda identificar a una persona específica, como el nombre, el número de identificación emitido por el gobierno, la fecha de nacimiento, la ocupación o la dirección.

La anonimización es una técnica de procesamiento de datos que elimina o modifica la PII. Genera como resultado
datos anónimos que no pueden asociarse a ninguna persona.

A continuación, vamos a ver ejemplos para proteger datos confidenciales y sensibles.


Primero vamos a generar datos falsos, por medio de la librería [faker](https://https://faker.readthedocs.io/en/master/), que contengan datos sensibles o confidenciales.

In [ ]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.7 MB/s eta 0:00:00


In [ ]:
!mkdir -p datalake/sensitive/personal_data/

In [ ]:
from faker import Faker
import pandas as pd
import random

# Inicializar la instancia de Faker
fake = Faker()

# Crear listas para almacenar los datos generados
num_records = 100  # Número de registros en el dataset
data = []

# Generar datos ficticios sensibles
for _ in range(num_records):
    full_name = fake.name()
    email = fake.email()
    phone_number = fake.msisdn()
    birthdate = fake.date_of_birth(minimum_age=18, maximum_age=80)
    credit_card_number = fake.credit_card_number()
    social_security_number = fake.ssn()

    data.append([full_name, email, phone_number, birthdate, credit_card_number, social_security_number])

# Crear un DataFrame con los datos generados
columns = ['full_name', 'email', 'phone_number', 'birth_date', 'credit_card_number', 'ssn']
df_sensitive = pd.DataFrame(data, columns=columns)

df_sensitive.to_csv("datalake/sensitive/personal_data/people.csv", index=None)

df_sensitive.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn
0,Samuel Cunningham,danielle88@example.com,0421233690086,1952-03-05,36431126920813,757-33-2783
1,Darrell Evans,amiller@example.net,6873328391053,1999-07-05,3587478130678470,424-43-5143
2,Dustin Williams,jonesshannon@example.net,6660848809040,1946-03-29,2298375943262595,555-47-3396
3,Troy Johnson,autumnlawrence@example.com,1669488738455,1952-10-11,3516106562537742,824-75-2693
4,Thomas Maldonado,teresa03@example.org,1063791655394,1970-02-05,4470228155092059,148-92-5105


## Enmascaramiento parcial

Consiste en ocultar una porción de los datos con caracteres genéricos. La parte que no se oculta, por lo general, contiene información genérica relevante para análisis. Por ejemplo:

- en el caso de un email, se puede mantener el dominio, lo que está después del @.
- en el caso de un nro. de teléfono, se puede mantener los primeros dígitos para identificar el código de país y de área.

Veamos unos ejemplos.

In [ ]:
def mask_phone_number(df, column_name, n=5):
    """
    Enmascara parcialmente los números de teléfono
    en la columna especificada del DataFrame.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna que se va a anonimizar.
        n (int): El número de dígitos a mantener.

    Returns:
        pd.DataFrame: El DataFrame modificado con los números de teléfono enmascarados.
    """
    try:
        # Mantener los primeros 5 dígitos, reemplazar el resto con 'XXX'
        df.loc[:, column_name] = df[column_name].str[:n] + 'XXXXXX'
        return df
    except KeyError:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        return df

def mask_credit_card(df, column_name, n=4):
    """
    Enmascara los números de tarjeta de crédito
    en la columna especificada del DataFrame.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna que se va a anonimizar.
        n (int): El número de dígitos a mantener, tanto al final como al comienzo

    Returns:
        pd.DataFrame: El DataFrame modificado con los números de tarjeta de crédito anonimizados.
    """
    try:
        # Mantener los últimos 4 dígitos, reemplazar el resto con 'XXX'
        df.loc[:, column_name] = df[column_name].str[:n] + '*********' + df[column_name].str[-n:]
        return df
    except KeyError:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        return df

In [ ]:
df_anonymized = df_sensitive.copy()

In [ ]:
df_anonymized = mask_phone_number(df_anonymized, "phone_number", n=6)

In [ ]:
df_anonymized = mask_credit_card(df_anonymized, "credit_card_number")

In [ ]:
df_sensitive.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn
0,Samuel Cunningham,danielle88@example.com,0421233690086,1952-03-05,36431126920813,757-33-2783
1,Darrell Evans,amiller@example.net,6873328391053,1999-07-05,3587478130678470,424-43-5143
2,Dustin Williams,jonesshannon@example.net,6660848809040,1946-03-29,2298375943262595,555-47-3396
3,Troy Johnson,autumnlawrence@example.com,1669488738455,1952-10-11,3516106562537742,824-75-2693
4,Thomas Maldonado,teresa03@example.org,1063791655394,1970-02-05,4470228155092059,148-92-5105


In [ ]:
df_anonymized.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn
0,Samuel Cunningham,danielle88@example.com,042123XXXXXX,1952-03-05,3643*********0813,757-33-2783
1,Darrell Evans,amiller@example.net,687332XXXXXX,1999-07-05,3587*********8470,424-43-5143
2,Dustin Williams,jonesshannon@example.net,666084XXXXXX,1946-03-29,2298*********2595,555-47-3396
3,Troy Johnson,autumnlawrence@example.com,166948XXXXXX,1952-10-11,3516*********7742,824-75-2693
4,Thomas Maldonado,teresa03@example.org,106379XXXXXX,1970-02-05,4470*********2059,148-92-5105


In [ ]:
def mask_email(df, column_name):
    """
    Enmascara las direcciones de correo electrónico en la columna especificada del DataFrame,
    manteniendo solo el dominio.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna que se va a anonimizar.

    Returns:
        pd.DataFrame: El DataFrame modificado con los dominios de correo electrónico anonimizados.
    """
    try:
        # Extraer el dominio de correo electrónico y reemplazar la columna
        df.loc[:, column_name] = '****@' + df[column_name].str.split('@').str[1]
        return df
    except KeyError:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        return df


In [ ]:
df_anonymized = mask_email(df_anonymized, "email")

In [ ]:
df_anonymized.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn
0,Samuel Cunningham,****@example.com,042123XXXXXX,1952-03-05,3643*********0813,757-33-2783
1,Darrell Evans,****@example.net,687332XXXXXX,1999-07-05,3587*********8470,424-43-5143
2,Dustin Williams,****@example.net,666084XXXXXX,1946-03-29,2298*********2595,555-47-3396
3,Troy Johnson,****@example.com,166948XXXXXX,1952-10-11,3516*********7742,824-75-2693
4,Thomas Maldonado,****@example.org,106379XXXXXX,1970-02-05,4470*********2059,148-92-5105


## Hashing

Una solución sencilla, al momento de trabajar con PII, es eliminar estos campos antes de compartir los datos. Sin embargo, en algunas ocasiones se necesita disponer de los datos de identificación personal. Por ejemplo, las empresas que comercializan servicios analizan que clientes tiene una probabilidad alta de cancelar su membresía o suscripción para poder ofrecerles descuentos o beneficios adicionales y retenerlos. En este caso, deben trabajar con información para identificar a cada cliente. Es posible anonimizar los campos con PII por medio de **hashing**.

El hashing es un proceso unidireccional de transformación de una cadena de caracteres de texto plano en una cadena única de longitud fija. El proceso de hashing tiene dos características importantes:
- Es muy difícil convertir un string "hasheado" en su forma original.
- La misma cadena de texto plano producirá el mismo resultado cifrado.

De esta forma, en vez de compartir campos con PII, vamos a compartir su versión "hasheada".

En el siguiente ejemplo, vamos a usar librería standard `hashlib` de Python.

In [ ]:
import hashlib
hashlib.sha256("Guido Franco")

TypeError: Strings must be encoded before hashing

In [ ]:
import hashlib
hashlib.sha256("Guido Franco".encode())

<sha256 _hashlib.HASH object @ 0x7a9356d98fb0>

In [ ]:
import hashlib
hashlib.sha256("Guido Franco".encode()).hexdigest()

'b278386c11070337516dc58cea2e48ebf4ba553fa422cff51d751779683d31cf'

In [ ]:
import hashlib
hashlib.sha256("GUido  Franco".encode()).hexdigest()

'a149eda316e4b77c2f51c4381ac8dcae3dac4f2287138dc33bd8a948fb2551f9'

In [ ]:
hashlib.sha256("GUIDO FRANCO".encode()).hexdigest()

'b8c657ad14b52b6f8dc6976e5408f527f102482f6b20414f9644466167081249'

In [ ]:
import hashlib
nombre = "Guido Franco"
nombre = nombre.lower()
hash_value_object = hashlib.sha256(nombre.encode())
hash_value = hash_value_object.hexdigest()
print(hash_value)

d615a5fc78a80330add0b5b84d09ba8ea4920138e1d746f0f5760e6b8767d76e


In [ ]:
import hashlib

def hash_column(df, column_name):
    """
    Aplica el algoritmo "Hash" a los valores en una columna del DataFrame.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna que se va a hashear.

    Returns:
        pd.DataFrame: El DataFrame modificado con los valores hasheados en la columna especificada.
    """
    try:

        # Crear una nueva columna _hashed con los valores hasheados de la columna recibida
        df[f"{column_name}_hashed"] = df[column_name].apply(
            lambda value: hashlib.sha256(value.encode()).hexdigest()
            )

        return df
    except KeyError:
        print(f"La columna '{column_name}' no existe en el DataFrame.")
        return df

In [ ]:
df_anonymized = hash_column(df_anonymized, "ssn")

In [ ]:
df_anonymized = hash_column(df_anonymized, "full_name")

In [ ]:
df_anonymized.head()

,full_name,email,phone_number,birth_date,credit_card_number,ssn,ssn_hashed,full_name_hashed
0,Samuel Cunningham,****@example.com,042123XXXXXX,1952-03-05,3643*********0813,757-33-2783,1230187eefd1f5317e74e2c03df9e9ab5721ee2af82b98...,a8825ea846ff7b797342146dd40e6a82ec5b705693d982...
1,Darrell Evans,****@example.net,687332XXXXXX,1999-07-05,3587*********8470,424-43-5143,14ee4bb0562bba145561c1e3e629e9e2e8c202ad539714...,1489fd9708f5b60a461b0411ff00b93d76ad7cb75ebc48...
2,Dustin Williams,****@example.net,666084XXXXXX,1946-03-29,2298*********2595,555-47-3396,ef27811cbbb71f4999d7f95c3da84c6f9926b6fb10fec0...,eedcd7c3ce8c434629bcf025d3072ee8ec965cc37e0c21...
3,Troy Johnson,****@example.com,166948XXXXXX,1952-10-11,3516*********7742,824-75-2693,5011fa310f2579112c2b0571a5a9bbf600ac51d7b61bb3...,21720025582b6d637d0a6620b5cf1ccdd0b88c0b494144...
4,Thomas Maldonado,****@example.org,106379XXXXXX,1970-02-05,4470*********2059,148-92-5105,f6a806452a999132d1c7e1c440d25c9133c9eb7c1534df...,b455e483318c9189d252a36bb9bf3d8a94a6854bf54fdf...


In [ ]:
df_anonymized.drop(columns=["full_name", "ssn"], inplace=True)

In [ ]:
df_anonymized.head()

,email,phone_number,birth_date,credit_card_number,ssn_hashed,full_name_hashed
0,****@example.com,042123XXXXXX,1952-03-05,3643*********0813,1230187eefd1f5317e74e2c03df9e9ab5721ee2af82b98...,a8825ea846ff7b797342146dd40e6a82ec5b705693d982...
1,****@example.net,687332XXXXXX,1999-07-05,3587*********8470,14ee4bb0562bba145561c1e3e629e9e2e8c202ad539714...,1489fd9708f5b60a461b0411ff00b93d76ad7cb75ebc48...
2,****@example.net,666084XXXXXX,1946-03-29,2298*********2595,ef27811cbbb71f4999d7f95c3da84c6f9926b6fb10fec0...,eedcd7c3ce8c434629bcf025d3072ee8ec965cc37e0c21...
3,****@example.com,166948XXXXXX,1952-10-11,3516*********7742,5011fa310f2579112c2b0571a5a9bbf600ac51d7b61bb3...,21720025582b6d637d0a6620b5cf1ccdd0b88c0b494144...
4,****@example.org,106379XXXXXX,1970-02-05,4470*********2059,f6a806452a999132d1c7e1c440d25c9133c9eb7c1534df...,b455e483318c9189d252a36bb9bf3d8a94a6854bf54fdf...


Se ha creado nuevas columnas: `ssn_hashed` y `full_name_hashed`. Es necesario borrar las columnas originales si se pretende compartir esta información, o bien reemplazarlo con su versión hasheada.

## Generalización
Hay ciertos elementos de datos que se relacionan más fácilmente con determinados individuos. Para protegerlos, utilizamos la generalización para **eliminar una parte** de los datos o **reemplazarlos por un valor común.**

A continuación, vamos a aplicar generalización sobre `birth_date`, eliminando el día y manteniendo solo el mes y el año.

> *La generalización nos permite lograr el k-anonimato (k-anonymity), un término estándar en la industria utilizado para describir una técnica para ocultar la identidad de individuos en un grupo de personas similares. En el anonimato k, k es un número que representa el tamaño de un grupo. Si para cualquier individuo del conjunto de datos, hay al menos k-1 individuos que tienen las mismas propiedades, entonces hemos conseguido el k-anonimato para el dataset. Por ejemplo, imaginemos un determinado dataset en el que k es igual a 50 y la propiedad es el código postal. Si observamos a cualquier persona dentro de ese conjunto de datos, siempre encontraremos a otras 49 con el mismo código postal. Por lo tanto, no podríamos identificar a ninguna persona sólo a partir de su código postal.*

*Mas info del k anonimato:*
- [Discover k-Anonymity, a Property of Anonymized Data](https://blog.pangeanic.com/discover-k-anonymity)

- [How Google anonymizes data](https://policies.google.com/technologies/anonymization?hl=en-US)


In [ ]:
def truncate_date_to_decade(df, column_name):
    """
    Convierte una columna a fecha al formato decada.

    Parameters:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        column_name (str): El nombre de la columna de fechas que se va a truncar.

    Returns:
        pd.DataFrame: El DataFrame modificado con los valores de la columna truncados al mes.
    """
    try:
        # Verificar si es un DataFrame
        if not isinstance(df, pd.DataFrame):
            raise ValueError("El argumento 'df' debe ser un DataFrame.")

        # Verificar si la columna existe
        if column_name not in df.columns:
            raise ValueError(f"La columna '{column_name}' no existe en el DataFrame.")

        # Verificar y convertir si es necesario
        if df[column_name].dtype != 'datetime64[ns]':
            try:
                df[column_name] = pd.to_datetime(df[column_name])
            except:
                raise ValueError(f"No se pudo convertir la columna '{column_name}' a tipo datetime.")

        # Convertir a decada
        df[column_name] = (df[column_name].dt.year // 10)*10
        return df
    except Exception as e:
        print(f"Error: {e}")
        return df

In [ ]:
df_anonymized = truncate_date_to_decade(df_anonymized, "birth_date")

In [ ]:
df_anonymized.head()

,email,phone_number,birth_date,credit_card_number,ssn_hashed,full_name_hashed
0,****@example.com,042123XXXXXX,1950,3643*********0813,1230187eefd1f5317e74e2c03df9e9ab5721ee2af82b98...,a8825ea846ff7b797342146dd40e6a82ec5b705693d982...
1,****@example.net,687332XXXXXX,1990,3587*********8470,14ee4bb0562bba145561c1e3e629e9e2e8c202ad539714...,1489fd9708f5b60a461b0411ff00b93d76ad7cb75ebc48...
2,****@example.net,666084XXXXXX,1940,2298*********2595,ef27811cbbb71f4999d7f95c3da84c6f9926b6fb10fec0...,eedcd7c3ce8c434629bcf025d3072ee8ec965cc37e0c21...
3,****@example.com,166948XXXXXX,1950,3516*********7742,5011fa310f2579112c2b0571a5a9bbf600ac51d7b61bb3...,21720025582b6d637d0a6620b5cf1ccdd0b88c0b494144...
4,****@example.org,106379XXXXXX,1970,4470*********2059,f6a806452a999132d1c7e1c440d25c9133c9eb7c1534df...,b455e483318c9189d252a36bb9bf3d8a94a6854bf54fdf...


Hemos llegado al final, has conocido y aplicado algunas técnicas para proteger y anonimizar datos con PII.

In [ ]:
!mkdir -p datalake/silver/personal_data
df_anonymized.to_csv("datalake/silver/personal_data/people.csv", index=None)

Hagamos de cuenta que somos Data Scientists

In [ ]:
df_people = pd.read_csv(
    "datalake/silver/personal_data/people.csv",
    usecols=["full_name_hashed", "ssn_hashed", "email", "phone_number"])
df_people.head()

,email,phone_number,ssn_hashed,full_name_hashed
0,****@example.com,042123XXXXXX,1230187eefd1f5317e74e2c03df9e9ab5721ee2af82b98...,a8825ea846ff7b797342146dd40e6a82ec5b705693d982...
1,****@example.net,687332XXXXXX,14ee4bb0562bba145561c1e3e629e9e2e8c202ad539714...,1489fd9708f5b60a461b0411ff00b93d76ad7cb75ebc48...
2,****@example.net,666084XXXXXX,ef27811cbbb71f4999d7f95c3da84c6f9926b6fb10fec0...,eedcd7c3ce8c434629bcf025d3072ee8ec965cc37e0c21...
3,****@example.com,166948XXXXXX,5011fa310f2579112c2b0571a5a9bbf600ac51d7b61bb3...,21720025582b6d637d0a6620b5cf1ccdd0b88c0b494144...
4,****@example.org,106379XXXXXX,f6a806452a999132d1c7e1c440d25c9133c9eb7c1534df...,b455e483318c9189d252a36bb9bf3d8a94a6854bf54fdf...


In [ ]:
import numpy as np

df_people["churn_probability"] = np.random.uniform(0, 1, df_people.shape[0])

In [ ]:
df_people_churn = df_people[df_people["churn_probability"] >= 0.8]

In [ ]:
df_people_churn.head()

,email,phone_number,ssn_hashed,full_name_hashed,churn_probability
4,****@example.org,106379XXXXXX,f6a806452a999132d1c7e1c440d25c9133c9eb7c1534df...,b455e483318c9189d252a36bb9bf3d8a94a6854bf54fdf...,0.964452
9,****@example.net,449974XXXXXX,afe01941311a6cf5dcbb076de6980be2e3881226926b20...,e4854261b1a82959d62c630928e452af2a936ef6930d9f...,0.819549
14,****@example.com,465684XXXXXX,c1321b4fe5185c1d6b7d0aa3f1352d2ae2ad56bdbe66fa...,20a1bade9850de841666fc37ebf972b31aff35167ba00d...,0.888389
15,****@example.com,719127XXXXXX,cdb07d96e521c1cb0b9e304e26a3de6816b492aba9b196...,9e41d18a259f282eb52db5ca0406513d8ef9b00c5f021f...,0.982047
17,****@example.com,302383XXXXXX,9085e294ca992597967ed76677f0175ef9895a79bc668f...,9530e2550d088098d3aeae8e35541b71cdc8daf2d18897...,0.981364


In [ ]:
df_people_sens = pd.read_csv("datalake/sensitive/personal_data/people.csv",
                             usecols=["full_name", "email", "phone_number", "ssn"])
df_people_sens.head()

,full_name,email,phone_number,ssn
0,Samuel Cunningham,danielle88@example.com,421233690086,757-33-2783
1,Darrell Evans,amiller@example.net,6873328391053,424-43-5143
2,Dustin Williams,jonesshannon@example.net,6660848809040,555-47-3396
3,Troy Johnson,autumnlawrence@example.com,1669488738455,824-75-2693
4,Thomas Maldonado,teresa03@example.org,1063791655394,148-92-5105


In [ ]:
df_people_sens = hash_column(df_people_sens.copy(), "ssn")
df_people_sens.head()

,full_name,email,phone_number,ssn,ssn_hashed
0,Samuel Cunningham,danielle88@example.com,421233690086,757-33-2783,1230187eefd1f5317e74e2c03df9e9ab5721ee2af82b98...
1,Darrell Evans,amiller@example.net,6873328391053,424-43-5143,14ee4bb0562bba145561c1e3e629e9e2e8c202ad539714...
2,Dustin Williams,jonesshannon@example.net,6660848809040,555-47-3396,ef27811cbbb71f4999d7f95c3da84c6f9926b6fb10fec0...
3,Troy Johnson,autumnlawrence@example.com,1669488738455,824-75-2693,5011fa310f2579112c2b0571a5a9bbf600ac51d7b61bb3...
4,Thomas Maldonado,teresa03@example.org,1063791655394,148-92-5105,f6a806452a999132d1c7e1c440d25c9133c9eb7c1534df...


In [ ]:
pd.merge(df_people_churn, df_people_sens, on="ssn_hashed", how="inner")

,email_x,phone_number_x,ssn_hashed,full_name_hashed,churn_probability,full_name,email_y,phone_number_y,ssn
0,****@example.org,106379XXXXXX,f6a806452a999132d1c7e1c440d25c9133c9eb7c1534df...,b455e483318c9189d252a36bb9bf3d8a94a6854bf54fdf...,0.964452,Thomas Maldonado,teresa03@example.org,1063791655394,148-92-5105
1,****@example.net,449974XXXXXX,afe01941311a6cf5dcbb076de6980be2e3881226926b20...,e4854261b1a82959d62c630928e452af2a936ef6930d9f...,0.819549,Steve Shah,dchase@example.net,4499742889478,830-50-8715
2,****@example.com,465684XXXXXX,c1321b4fe5185c1d6b7d0aa3f1352d2ae2ad56bdbe66fa...,20a1bade9850de841666fc37ebf972b31aff35167ba00d...,0.888389,Kathryn Butler,csims@example.com,4656847623888,862-19-1488
3,****@example.com,719127XXXXXX,cdb07d96e521c1cb0b9e304e26a3de6816b492aba9b196...,9e41d18a259f282eb52db5ca0406513d8ef9b00c5f021f...,0.982047,Mary Morgan,sharoncaldwell@example.com,7191277834866,718-54-0020
4,****@example.com,302383XXXXXX,9085e294ca992597967ed76677f0175ef9895a79bc668f...,9530e2550d088098d3aeae8e35541b71cdc8daf2d18897...,0.981364,Laura Snyder,charles99@example.com,3023833868221,460-63-5303
5,****@example.net,747526XXXXXX,6641607e65ab506dbec3c7b3b21038104848378cdf98f8...,895c0a3c66b4e9c24cbe5845a5067394cdac52dfbd8610...,0.948632,Michelle Clark,jill43@example.net,7475265106001,371-90-3514
6,****@example.org,972590XXXXXX,0ad8549bdb0a1e3926fa76fdc6bf0ae226bec42927bb6c...,3614d1ebad7c5cef2a137c17c9fc4fff347a28e0f433ff...,0.806813,Shawn Reeves,kellygould@example.org,9725904234919,459-03-1837
7,****@example.net,086647XXXXXX,dcf7d6874d9ecf3d0bb35d8c795e937d4c13ca087eb964...,bdb473e2501b0139ccd9399eda263374146352fa5969c5...,0.970202,Margaret Clark,james62@example.net,866472879825,564-45-6397
8,****@example.org,427092XXXXXX,97a9bb9deb39f42a369b4effb69535ec8a8ad656395a1d...,d8b416e942182df3f7d3887c1d23a472b504b1e072a921...,0.841920,Preston Morgan,sanfordsarah@example.org,4270927650188,879-36-3748
9,****@example.net,641222XXXXXX,936302092cf932242b1efe43cb4a8c2d54f7c56833e0fd...,1f5866b2a3161ae666b322a8ea50feb5128e5b01fb63cf...,0.931621,Ricky Parker,fgraham@example.net,6412223985325,093-96-0345
